In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'

# # dx = 1 km; Np = 1M; Nt = 5 min
# data=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_1km_1e6.nc') #***
# parcel=xr.open_dataset(dir+'../cm1r20.3/run/cm1out_pdata_1km_1e6.nc') #***
# res='1km'
# Np_str='1e6'

# dx = 1km; Np = 50M
#Importing Model Data
check=False
dir2='/home/air673/koa_scratch/'
data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_50M.nc') #***
res='1km'; t_res='1min'; Np_str='50e6'

# # dx = 1km; Np = 100M
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_1km_1min.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_1km_1min_100M.nc') #***
# res='1km'; t_res='1min'; Np_str='100e6'


# dx = 250 m
# #Importing Model Data
# check=False
# dir2='/home/air673/koa_scratch/'
# data=xr.open_dataset(dir2+'cm1out_250m.nc') #***
# parcel=xr.open_dataset(dir2+'cm1out_pdata_250m.nc') #***

In [ ]:
import sys
dir2='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
path=dir2+'../Functions/'
sys.path.append(path)

import NumericalFunctions
from NumericalFunctions import * # import NumericalFunctions 
import PlottingFunctions
from PlottingFunctions import * # import PlottingFunctions


# # Get all functions in NumericalFunctions
# import inspect
# functions = [f[0] for f in inspect.getmembers(NumericalFunctions, inspect.isfunction)]
# functions

In [ ]:
# # #TESTING W (T,Z) PLOT
# # w_tz=data['winterp'].mean(dim=("xh",'yh'))

# # w_masked = data['winterp'].where((data['winterp'] >= 0.1) & (data['qc'] + data['qi'] >= 1e-6))
# # w_tz_A = w_masked.mean(dim=("xh", "yh"))  # Compute horizontal mean ignoring NaNs

# # New Subplots for Contour Plots
# fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# # First contour plot
# cf1 = axes[0].contourf(w_tz.T)
# plt.colorbar(cf1, ax=axes[0],label='m/s')
# axes[0].set_ylabel('zgrid')
# axes[0].set_xlabel('time')
# axes[0].set_title('W(z,t) Horizontal Average for Eulerian Data')

# # Second contour plot
# cf2 = axes[1].contourf(w_tz_A.T)
# plt.colorbar(cf2, ax=axes[1],label='m/s')
# axes[1].set_ylabel('zgrid')
# axes[1].set_xlabel('time')
# axes[1].set_title('W(z,t) Horizontal Average for Eulerian Data (w ≥ 0.1) & (qc+qi ≥ 1e-6)')
# plt.tight_layout()

In [ ]:
# #TESTING W (T,Z) PLOT

# z_lev=10
# #GETTING W AND A AT Z LEVEL
# W_z = W[np.where(Z==15)]
# A_z = A_c[np.where(Z==15)]

# #REMOVING ZERO
# W_z = W_z[np.where(A_z!=0)]
# A_z = A_z[np.where(A_z!=0)]

# fig, axes = plt.subplots(1, 2, figsize=(10, 4))
# # First subplot
# axes[0].hist(W_z, bins=30, color='blue', alpha=0.7)
# axes[0].set_title(f"W (z = {data['zh'][z_lev].values*1e3:.0f} m)")
# # Second subplot
# axes[1].hist([a * w for a, w in zip(A_z, W_z)], bins=30, color='blue', alpha=0.7)
# axes[1].set_title(f"W * A (z = {data['zh'][z_lev].values*1e3:.0f} m)")

# axes[0].set_ylabel('count');axes[1].set_ylabel('count')
# axes[0].set_xlabel('W (m/s)');axes[1].set_xlabel('W*A (m/s)')
# plt.suptitle('Plotting W array as well as W*A_c for eulerian W for lagrangian parcel at a specific time')
# plt.tight_layout()
# # Lx=512*1000;Ly=200*1000
# # np.sum(A_z*W_z)*np.mean(m_arr)/Lx/Ly/(data['zh'][z_lev].values-data['zh'][z_lev-1].values)/1000


In [ ]:
########################################################
#PLOTTING
plotting=False #KEEP FALSE WHEN JOB ARRAY IS RUNNING
plotting=True

In [ ]:
if plotting==True:
    #constants
    Cp=1004 #Jkg-1K-1
    Cv=717 #Jkg-1K-1
    Rd=Cp-Cv #Jkg-1K-1
    eps=0.608
    
    Lx=(data['xf'][-1].item()-data['xf'][0].item())*1000 #x length (m)
    Ly=(data['yf'][-1].item()-data['yf'][0].item())*1000 #y length (m)
    Np=len(parcel['xh']) #number of lagrangian parcles
    dt=(data['time'][1]-data['time'][0]).item()/1e9 #sec
    dx=(data['xf'][1].item()-data['xf'][0].item())*1e3 #meters
    dy=(data['yf'][1].item()-data['yf'][0].item())*1e3 #meters
    xs=data['xf'].values*1000
    ys=data['yf'].values*1000
    zs=data['zf'].values*1000
    
    def zf(z):
        k=z #z is the # level of z
        out=data['zf'].values[k]*1000
        
        return out
    # def rho(x,y,z,t):
    #     p=data['prs'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     p0=101325 #Pa
    #     theta=data['th'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     T=theta*(p/p0)**(Rd/Cp)
    #     qv=data['qv'].isel(xh=x,yh=y,zh=z,time=t).item()
    #     # Tv=T*(1+eps*qv)
    #     Tv=T*(eps+qv)/(eps*(1+qv))
    #     rho = p/(Rd*Tv)
    #     out=rho
    #     return out
    
    def rho(x,y,z,rho_data_t):
        out=rho_data_t[z,y,x]
        return out
    def m(t):
        rho_data_t=data['rho'].isel(time=t).data
        
        m=0
        #triple sum
        for k in range(len(data['zh'])):
            dz=(zf(k+1)-zf(k))
            for j in range(len(data['yh'])):
                for i in range(len(data['xh'])):
                    rho_out=rho(i,j,k,rho_data_t)
                    m+=rho_out*dz
                    
        #triple sum
        out=m*dx*dy/Np
        return out


In [ ]:
if plotting==True:
    #Calculate Mass Constant
    # calculate='single_time'
    # calculate=True
    calculate=False
    
    if calculate==True:
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
        for t in np.arange(Nt):
            if np.mod(t,25)==0: print(t)
            m_arr[t]=m(t)
        dir3=dir+f'Project_Algorithms/Entrainment/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    elif calculate=='single_time':
        Nt=len(data['time'])
        m_arr=np.zeros((Nt))
    
        t=0 #len(data['time'])//2 #Pick some middle time
        m_300=m(t)
        for t in np.arange(Nt):
            m_arr[t]=m_300 #UNCOMMENT FOR FULL CALCULATION
        dir3=dir+f'Project_Algorithms/Entrainment/'
        np.save(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy', m_arr)
    else:
        dir3=dir+f'Project_Algorithms/Entrainment/'
        m_arr = np.load(dir3+f'Mass_Array_{res}_{t_res}_{Np_str}.npy')
    
    # # TESTING
    # lst=[]
    # for t in np.arange(133):
    #     lst.append(m_arr[t])
    
    # plt.plot(lst)
    # (np.max(lst)-np.min(lst))*100/np.mean(lst)

In [ ]:
# #NONOPTIMIZED LOADING AND AVERAGING (NOT RECOMMENDED)
# PROCESSING=False
# # PROCESSING=True

# if PROCESSING==False:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_VMF_profiles_{res}_{t_res}_{Np_str}.h5'
# if PROCESSING==True:
#     dir3=dir+f'Project_Algorithms/Entrainment/3D_VMF_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
# with h5py.File(dir3, "r") as h5f:
#     profile_array_VMF_g = h5f["profile_array_VMF_g"][:]
#     profile_array_VMF_c = h5f["profile_array_VMF_c"][:]

# def apply_constant(profile_array,apply):
#     if apply==True:
#         Nt=profile_array.shape[0]
#         Nz=profile_array.shape[1]
    
#         profile_array/=(dx*dy)
#         for t in np.arange(Nt):
#             profile_array[t]*=m_arr[t]
#         for z in np.arange(Nz):
#             dz=zf(z+1)-zf(z)
#             profile_array[:,z]/=dz
#     return profile_array

# profile_array_VMF_g=apply_constant(profile_array_VMF_g,apply=True)
# profile_array_VMF_c=apply_constant(profile_array_VMF_c,apply=True)

In [ ]:
if plotting==True:
    #OPTIMIZED LOADING AND AVERAGING
    def apply_constant_tbyt(profile_array,t,apply):
        if apply==True:
            Nt=len(data['time'])
            Nz=len(data['zh'])
        
            profile_array/=(dx*dy)
            profile_array*=m_arr[t]
            for z in np.arange(Nz):
                dz=zf(z+1)-zf(z)
                profile_array[z]/=dz
        return profile_array
    
    
    PROCESSING=False
    # PROCESSING=True
    
    if PROCESSING==False:
        dir3=dir+f'Project_Algorithms/Entrainment/3D_VMF_profiles_{res}_{t_res}_{Np_str}.h5'
    if PROCESSING==True:
        dir3=dir+f'Project_Algorithms/Entrainment/3D_VMF_profiles_PREPROCESSING_{res}_{t_res}_{Np_str}.h5'
    
    def load_get_mean(type):
        Nt=len(data['time']); Nz=len(data['zh']); 
        VMF_output_array = np.zeros((Nt, Nz))
    
        if type=='general':
            var_string="profile_array_VMF_g"
        elif type=='cloudy':
            var_string="profile_array_VMF_c"
    
        
        with h5py.File(dir3, "r") as h5f:
            #Reading
    
            for t in np.arange(Nt):
                if np.mod(t,50)==0: print(t)
                profile_array_VMF = h5f[var_string][t]
        
                #Applying Constants
                profile_array_VMF=apply_constant_tbyt(profile_array_VMF,t,apply=True)
        
    
                VMF_mean_yx=np.mean(profile_array_VMF, axis = (1,2))
        
    
                VMF_output_array[t]=VMF_mean_yx
        
        return VMF_output_array
    
    
    # type='general'
    type='cloudy'
    
    profile_array_VMF_c = load_get_mean(type)

In [ ]:
if plotting==True:
    # type='general'
    type='cloudy'
    
    print('taking horizontal mean')
    profile_array=profile_array_VMF_c.copy()
    
    
    import matplotlib.pyplot as plt
    from matplotlib.gridspec import GridSpec
    import numpy as np
    
    fig = plt.figure(figsize=(10, 8))
    gs = GridSpec(2, 2, figure=fig)
    
    ######
    cmap1 = plt.cm.viridis
    cmap2 = plt.cm.seismic 
    n_levels=29
    ######
    
    # ######
    # vmax_shared = np.max([np.max(profile_array_e), np.max(profile_array_d)])
    # norm_shared = mcolors.Normalize(vmin=0, vmax=vmax_shared)
    # ######
    
    # First subplot: VMF
    ########################################
    ax1 = fig.add_subplot(gs[0, 0])
    # contour1 = ax1.contourf(profile_array_e.T, cmap=cmap1)
    contour1 = ax1.contourf(profile_array.T, cmap=cmap1)
    cbar1=fig.colorbar(contour1, ax=ax1)
    apply_scientific_notation_colorbar([cbar1])
    Nz = len(data['zh'])
    ax1.set_yticks(np.arange(Nz))
    new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
    ax1.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
    ax1.set_ylabel('z (km)');ax1.set_xlabel('t (timesteps)')
    ax1.set_title('Entrainment using Lagrangian Binary Array',fontsize=8)
    
    # #FIXING TICKS
    # ax3.set_yticks(np.arange(Nz))
    # new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
    # ax3.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
    # ax3.set_ylabel('z (km)');ax3.set_xlabel('t (timesteps)')
    # ax3.set_title('Entrainment - Detrainment')
    
    # #FIXING SCIENTIFIC NOTATION
    # from matplotlib.ticker import ScalarFormatter
    # formatter = ScalarFormatter(useMathText=True)
    # formatter.set_powerlimits((-2, 2))  # Adjust the range for scientific notation
    # for cbar in (cbar1,cbar2, cbar3):  # These must be Colorbar instances
    #     cbar.formatter = formatter
    #     cbar.update_ticks()
    
    # Display the plot
    plt.tight_layout()

In [ ]:
if plotting==True:
    plt.plot(np.mean(profile_array,axis=(0)),data['zh'],label='VMF')
    
    plt.legend(); plt.title('2D Vertical Mass Flux Using Lagrangian Binary Array')
    
    from matplotlib.ticker import ScalarFormatter
    formatter = ScalarFormatter(useMathText=True)
    formatter.set_scientific(True)
    formatter.set_powerlimits((-1, 1))
    plt.gca().xaxis.set_major_formatter(formatter)
    
    plt.ylabel('z (km)');plt.xlabel('Vertical Mass Flux, M(t,z) ' + r'$(kg m^{-2} s^{-1})$')
    
    # #COMPARING MAXIMUM WITH ROMPS 2012
    # (1.4e-2)/(4e-3)

In [ ]:
# #TESTING DISTRIBUTION

# import matplotlib.pyplot as plt

# fig, axes = plt.subplots(1, 3, figsize=(15, 5))  # Create a 1-row, 3-column subplot

# array_z=np.mean(profile_array,axis=(0)); where=np.where(array_z==np.max(array_z))[0] #12
# profile_array_z=profile_array.copy()[:,where]

# # Plot the first histogram for all values
# axes[0].hist(profile_array_z)
# axes[0].set_ylabel('Count')
# axes[0].set_xlabel('Vertical Mass Flux, M(z) ' + r'$(kg m^{-2} s^{-1})$')
# axes[0].set_title('Histogram of VMF(t,z)')

# # Threshold-based filtering
# threshold = 0.01  
# filtered_data_below = profile_array_z[profile_array_z < threshold]
# filtered_data_above = profile_array_z[profile_array_z > threshold]

# # Plot the second histogram for values < 0.01
# axes[1].hist(filtered_data_below)
# axes[1].set_ylabel('Count')
# axes[1].set_xlabel('Vertical Mass Flux, M(z) ' + r'$(kg m^{-2} s^{-1})$')
# axes[1].set_title('Histogram of VMF(t,z) for values < 0.01')

# # Plot the third histogram for values > 0.01
# axes[2].hist(filtered_data_above)
# axes[2].set_ylabel('Count')
# axes[2].set_xlabel('Vertical Mass Flux, M(z) ' + r'$(kg m^{-2} s^{-1})$')
# axes[2].set_title('Histogram of VMF(t,z) for values > 0.01')

# plt.suptitle(f"Histograms of VMF at z = {data['zh'].values[where]} km (the location of the peak in VMF(t,z)")
# plt.tight_layout()  # Adjust layout to prevent overlap

# f"{(0.0005*50)*100/(0.03*14):.2f}% contribution for the values below 0.001"